# Complex Query Full-Trace Monitoring with GPT-4

Navigation links:
- [Reading guide](../READING_GUIDE.md)
- [Project README](../README.md)
- [Architecture guide](../AGENT_FRAMEWORK_ARCHITECTURE.md)
- [Tracing plan](../TRAINING_DATA_TRACING_PLAN.md)

This notebook is built for **full observability**:
1. Run a complex query that requires planning.
2. Inspect full LLM input/output (system prompt, user prompt, raw response, parsed JSON, latency).
3. Inspect full agent loop timeline and saved trace artifacts (`session.json`, `events.jsonl`).


## 1) Setup and Environment Checks

Requirements:
- `.env` with `OPENAI_API_KEY`
- Internet connection for real tools


In [1]:
from __future__ import annotations

import json
import os
import re
import sys
import time
from datetime import datetime, timezone
from pathlib import Path
from typing import Any

from dotenv import load_dotenv

cwd = Path.cwd().resolve()
PROJECT_ROOT = cwd if (cwd / "src").exists() else cwd.parent
if not (PROJECT_ROOT / "src").exists():
    raise RuntimeError("Could not locate project root with src/ directory")

if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / "src"))

load_dotenv(PROJECT_ROOT / ".env")

API_KEY = (os.getenv("OPENAI_API_KEY") or "").strip()
MODEL_NAME = os.getenv("OPENAI_MODEL", "gpt-4")

print("Project root:", PROJECT_ROOT)
print("OPENAI_API_KEY detected:", bool(API_KEY))
print("OPENAI_MODEL:", MODEL_NAME)

if not API_KEY:
    raise RuntimeError("OPENAI_API_KEY is required for this notebook.")


Project root: /Users/admin/TuanDung/paper_implementation/plan_and_act_repro
OPENAI_API_KEY detected: True
OPENAI_MODEL: gpt-4


## 2) Build Instrumented LLM Client

This client logs all model calls with:
- component name (`planner`, `executor`, `replanner`)
- prompts
- raw text response
- parsed JSON output
- latency and status


In [3]:
from openai import BadRequestError, OpenAI


class InstrumentedLLMClient:
    def __init__(self, *, component: str, ledger: list[dict[str, Any]], api_key: str, base_url: str = "") -> None:
        self.component = component
        self.ledger = ledger
        self.api_key = api_key
        self.base_url = base_url.strip()

    @property
    def enabled(self) -> bool:
        return bool(self.api_key)

    def _client(self) -> OpenAI:
        kwargs: dict[str, Any] = {"api_key": self.api_key}
        if self.base_url:
            kwargs["base_url"] = self.base_url
        return OpenAI(**kwargs)

    @staticmethod
    def _parse_json_content(content: str) -> dict[str, Any]:
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            pass

        fenced = re.search(r"```(?:json)?\s*(\{.*\})\s*```", content, flags=re.DOTALL | re.IGNORECASE)
        if fenced:
            return json.loads(fenced.group(1))

        start = content.find("{")
        end = content.rfind("}")
        if start != -1 and end != -1 and end > start:
            return json.loads(content[start : end + 1])

        raise ValueError("Model output is not valid JSON")

    def _repair_json(self, *, model: str, raw_text: str) -> tuple[str, dict[str, Any]]:
        fixer_system = "You convert malformed model output into strict JSON. Return JSON only."
        fixer_user = (
            "Repair this into a valid JSON object with no markdown and no explanation.\n"
            f"RAW:\n{raw_text}"
        )
        resp = self._client().chat.completions.create(
            model=model,
            temperature=0,
            messages=[
                {"role": "system", "content": fixer_system},
                {"role": "user", "content": fixer_user},
            ],
        )
        repaired_raw = resp.choices[0].message.content or "{}"
        parsed = self._parse_json_content(repaired_raw)
        return repaired_raw, parsed

    def chat_json(
        self,
        *,
        model: str,
        system_prompt: str,
        user_prompt: str,
        temperature: float,
    ) -> dict[str, Any]:
        if not self.enabled:
            raise RuntimeError("OPENAI_API_KEY is not set")

        started = time.perf_counter()
        status = "ok"
        raw_response = ""
        parsed: dict[str, Any] | None = None
        error_message = ""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        try:
            client = self._client()
            try:
                resp = client.chat.completions.create(
                    model=model,
                    temperature=temperature,
                    response_format={"type": "json_object"},
                    messages=messages,
                )
                raw_response = resp.choices[0].message.content or "{}"
            except BadRequestError as exc:
                if "response_format" not in str(exc):
                    raise
                resp = client.chat.completions.create(
                    model=model,
                    temperature=temperature,
                    messages=messages,
                )
                raw_response = resp.choices[0].message.content or "{}"

            try:
                parsed = self._parse_json_content(raw_response)
            except Exception:
                repaired_raw, repaired_parsed = self._repair_json(model=model, raw_text=raw_response)
                raw_response = repaired_raw
                parsed = repaired_parsed

        except Exception as exc:
            status = "error"
            error_message = f"{type(exc).__name__}: {exc}"

        latency_ms = round((time.perf_counter() - started) * 1000, 2)

        self.ledger.append(
            {
                "timestamp": datetime.now(timezone.utc).isoformat(),
                "component": self.component,
                "model": model,
                "temperature": temperature,
                "status": status,
                "latency_ms": latency_ms,
                "system_prompt": system_prompt,
                "user_prompt": user_prompt,
                "raw_response": raw_response,
                "parsed_output": parsed,
                "error": error_message,
            }
        )

        if status != "ok" or parsed is None:
            raise RuntimeError(error_message or "Unknown LLM error")

        return parsed


## 3) Initialize Agents, Environment, and Tracer

- GPT-4 primary agents
- Heuristic fallbacks for notebook stability
- Tool environment for real actions
- Trace collector for full event timeline


In [4]:
from plan_and_act.agents.executor import ExecutorAgent
from plan_and_act.agents.planner import PlannerAgent
from plan_and_act.agents.replanner import ReplannerAgent
from plan_and_act.core.schemas import PlanStep
from plan_and_act.core.types import ModelConfig
from plan_and_act.environments.factory import build_environment
from plan_and_act.prompts.templates import PromptTemplates
from plan_and_act.tracing import TraceCollector, TraceConfig

llm_ledger: list[dict[str, Any]] = []

prompts = PromptTemplates(config_dir=str(PROJECT_ROOT / "configs" / "prompts"))
openai_cfg = ModelConfig(provider="openai", model=MODEL_NAME, temperature=0.0)
heur_cfg = ModelConfig(provider="heuristic", model=MODEL_NAME, temperature=0.0)

planner = PlannerAgent(openai_cfg, prompts)
executor = ExecutorAgent(openai_cfg, prompts)
replanner = ReplannerAgent(openai_cfg, prompts)

planner_fallback = PlannerAgent(heur_cfg, prompts)
executor_fallback = ExecutorAgent(heur_cfg, prompts)
replanner_fallback = ReplannerAgent(heur_cfg, prompts)

planner.llm = InstrumentedLLMClient(component="planner", ledger=llm_ledger, api_key=API_KEY)
executor.llm = InstrumentedLLMClient(component="executor", ledger=llm_ledger, api_key=API_KEY)
replanner.llm = InstrumentedLLMClient(component="replanner", ledger=llm_ledger, api_key=API_KEY)

env = build_environment("tool")

trace_run_id = "nbtrace_" + datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
trace_cfg = TraceConfig(enabled=True, base_dir=str(PROJECT_ROOT / "data" / "raw" / "traces"), flush_every=1)
tracer = TraceCollector(config=trace_cfg, run_id=trace_run_id)

print("Trace run id:", trace_run_id)
print("Environment:", env.name)


Trace run id: nbtrace_20260219T093108Z
Environment: tool_calling


## 4) Run Complex Planning Loop with Full Monitoring

Complex query:
- multi-step web research
- comparison
- weighted scoring
- final recommendation with evidence URLs


In [5]:
complex_goal = (
    "Research 3 recent long-horizon LLM agent approaches related to Plan-and-Act, "
    "compare them on novelty, reproducibility, and open-source readiness, then compute "
    "weighted score = 0.5*novelty + 0.3*reproducibility + 0.2*oss_readiness for each, "
    "and recommend the top approach with evidence URLs."
)

runtime_summary: dict[str, Any] = {
    "goal": complex_goal,
    "iterations": [],
    "fallbacks": [],
}

tracer.start_session(
    goal=complex_goal,
    environment={"kind": "tool", "name": env.name},
    model_stack={
        "planner": openai_cfg.model_dump(),
        "executor": openai_cfg.model_dump(),
        "replanner": openai_cfg.model_dump(),
    },
    runtime_config={"max_iters": 4, "dynamic_replanning": True},
    metadata={"source": "notebook"},
)

observation = env.reset(goal=complex_goal)
action_history: list[dict[str, Any]] = []
max_iters = 4
step_count = 0
success = False
final_answer = ""


def call_with_fallback(label: str, primary_fn, fallback_fn):
    try:
        out = primary_fn()
        return out, False, ""
    except Exception as exc:
        fb = fallback_fn()
        return fb, True, f"{label} failed: {type(exc).__name__}: {exc}"


# Initial planning
tracer.log_event(event_type="planner_input", step=step_count, payload={"goal": complex_goal, "observation": observation})
plan, used_fb, fb_reason = call_with_fallback(
    "planner",
    lambda: planner.plan(goal=complex_goal, observation=observation, action_history=action_history, use_cot=False),
    lambda: planner_fallback.plan(goal=complex_goal, observation=observation, action_history=action_history, use_cot=False),
)
if used_fb:
    runtime_summary["fallbacks"].append(fb_reason)
tracer.log_event(event_type="planner_output", step=step_count, payload={"steps": [s.model_dump() for s in plan.steps], "fallback": used_fb})

for _ in range(max_iters):
    if not plan.steps:
        break

    step_count += 1
    current_step = plan.steps[0]

    tracer.log_event(
        event_type="executor_input",
        step=step_count,
        payload={
            "current_step": current_step.model_dump(),
            "observation": observation,
            "action_history_length": len(action_history),
        },
    )

    action, used_fb_exec, fb_reason_exec = call_with_fallback(
        "executor",
        lambda: executor.act(
            goal=complex_goal,
            current_step=current_step,
            observation=observation,
            step_index=0,
            total_steps=max(1, len(plan.steps)),
            use_cot=False,
        ),
        lambda: executor_fallback.act(
            goal=complex_goal,
            current_step=current_step,
            observation=observation,
            step_index=0,
            total_steps=max(1, len(plan.steps)),
            use_cot=False,
        ),
    )
    if used_fb_exec:
        runtime_summary["fallbacks"].append(fb_reason_exec)

    tracer.log_event(event_type="executor_output", step=step_count, payload={"action": action.model_dump(), "fallback": used_fb_exec})

    env_result = env.step(action=action, step_count=step_count)
    tracer.log_event(
        event_type="environment_step",
        step=step_count,
        payload={
            "observation": env_result.observation,
            "done": env_result.done,
            "success": env_result.success,
            "final_answer": env_result.final_answer,
            "notes": env_result.notes,
        },
    )

    action_history.append(action.model_dump())
    runtime_summary["iterations"].append(
        {
            "step": step_count,
            "plan_step": current_step.model_dump(),
            "action": action.model_dump(),
            "observation_after": env_result.observation,
            "done": env_result.done,
            "success": env_result.success,
        }
    )

    if action.is_final or env_result.done:
        success = bool(action.is_final or env_result.success)
        final_answer = action.final_answer or env_result.final_answer
        break

    observation = env_result.observation

    tracer.log_event(
        event_type="replanner_input",
        step=step_count,
        payload={
            "previous_plan": [s.model_dump() for s in plan.steps],
            "action_history_length": len(action_history),
            "observation": observation,
        },
    )

    plan, used_fb_rep, fb_reason_rep = call_with_fallback(
        "replanner",
        lambda: replanner.replan(
            goal=complex_goal,
            previous_plan=[s.model_dump() for s in plan.steps],
            action_history=action_history,
            observation=observation,
            use_cot=False,
        ),
        lambda: replanner_fallback.replan(
            goal=complex_goal,
            previous_plan=[s.model_dump() for s in plan.steps],
            action_history=action_history,
            observation=observation,
            use_cot=False,
        ),
    )
    if used_fb_rep:
        runtime_summary["fallbacks"].append(fb_reason_rep)

    tracer.log_event(event_type="replanner_output", step=step_count, payload={"steps": [s.model_dump() for s in plan.steps], "fallback": used_fb_rep})

runtime_summary["success"] = success
runtime_summary["final_answer"] = final_answer
runtime_summary["trace_run_id"] = trace_run_id
runtime_summary["llm_call_count"] = len(llm_ledger)

tracer.log_event(
    event_type="episode_end",
    step=step_count,
    payload={
        "success": success,
        "final_answer": final_answer,
        "fallback_count": len(runtime_summary["fallbacks"]),
    },
)
tracer.close(
    status="completed",
    summary={
        "success": success,
        "step_count": step_count,
        "llm_call_count": len(llm_ledger),
        "fallback_count": len(runtime_summary["fallbacks"]),
    },
)

runtime_summary


{'goal': 'Research 3 recent long-horizon LLM agent approaches related to Plan-and-Act, compare them on novelty, reproducibility, and open-source readiness, then compute weighted score = 0.5*novelty + 0.3*reproducibility + 0.2*oss_readiness for each, and recommend the top approach with evidence URLs.',
 'iterations': [{'step': 1,
   'plan_step': {'step_id': 1,
    'intent': "Use 'web_search' tool to find recent long-horizon LLM agent approaches related to Plan-and-Act.",
    'success_criteria': '3 recent long-horizon LLM agent approaches have been found.'},
   'action': {'action_type': 'search',
    'target': 'web_search',
    'arguments': {'query': 'recent long-horizon LLM agent approaches related to Plan-and-Act'},
    'rationale': 'To find the recent long-horizon LLM agent approaches related to Plan-and-Act, a web search is necessary.',
    'is_final': False,
    'final_answer': ''},
   'observation_after': 'Step 1: Tool[web_search] returned: {"ok": true, "query": "recent long-horizo

## 5) Inspect Full LLM I/O Ledger

You can inspect every LLM call in detail, including prompts and raw outputs.


In [6]:
import pandas as pd
from IPython.display import display

ledger_df = pd.DataFrame(llm_ledger)
if ledger_df.empty:
    print("No LLM calls captured.")
else:
    display(
        ledger_df[["component", "model", "status", "latency_ms"]]
        .assign(call_index=lambda d: d.index)
        [["call_index", "component", "model", "status", "latency_ms"]]
    )


def show_call(call_index: int) -> None:
    row = llm_ledger[call_index]
    print(f"===== CALL {call_index} | {row['component']} | status={row['status']} =====")
    print("\n--- SYSTEM PROMPT ---\n")
    print(row["system_prompt"])
    print("\n--- USER PROMPT ---\n")
    print(row["user_prompt"])
    print("\n--- RAW RESPONSE ---\n")
    print(row["raw_response"])
    print("\n--- PARSED OUTPUT ---\n")
    print(json.dumps(row["parsed_output"], indent=2, ensure_ascii=False) if row["parsed_output"] else None)
    if row.get("error"):
        print("\n--- ERROR ---\n")
        print(row["error"])


# Inspect first call by default
if llm_ledger:
    show_call(0)


,call_index,component,model,status,latency_ms
0,0,planner,gpt-4,ok,9303.75
1,1,executor,gpt-4,ok,3148.64
2,2,replanner,gpt-4,ok,16809.05
3,3,executor,gpt-4,ok,6442.72
4,4,replanner,gpt-4,ok,19106.61
5,5,executor,gpt-4,ok,4375.88
6,6,replanner,gpt-4,ok,13949.85
7,7,executor,gpt-4,ok,3689.89
8,8,replanner,gpt-4,ok,19917.88


===== CALL 0 | planner | status=ok =====

--- SYSTEM PROMPT ---

You are the Planner. Convert user goals into concise, executable, high-level steps.
Return strictly valid JSON following the provided schema.

Output JSON schema: {"goal": str, "steps": [{"step_id": int, "intent": str, "success_criteria": str}]}

--- USER PROMPT ---

Goal: Research 3 recent long-horizon LLM agent approaches related to Plan-and-Act, compare them on novelty, reproducibility, and open-source readiness, then compute weighted score = 0.5*novelty + 0.3*reproducibility + 0.2*oss_readiness for each, and recommend the top approach with evidence URLs.
Observation: Tool environment initialized for goal: Research 3 recent long-horizon LLM agent approaches related to Plan-and-Act, compare them on novelty, reproducibility, and open-source readiness, then compute weighted score = 0.5*novelty + 0.3*reproducibility + 0.2*oss_readiness for each, and recommend the top approach with evidence URLs.. Registered tools=['calcula

## 6) Inspect Full Runtime Timeline from Trace Files

This section reads `session.json` and `events.jsonl` generated in this notebook run.


In [7]:
trace_dir = PROJECT_ROOT / "data" / "raw" / "traces" / trace_run_id
session_path = trace_dir / "session.json"
events_path = trace_dir / "events.jsonl"

print("Trace directory:", trace_dir)
print("session exists:", session_path.exists())
print("events exists:", events_path.exists())

session = json.loads(session_path.read_text(encoding="utf-8"))
print("\nSession summary:")
print(json.dumps(session.get("summary", {}), indent=2))

events = []
for line in events_path.read_text(encoding="utf-8").splitlines():
    line = line.strip()
    if line:
        events.append(json.loads(line))

events_df = pd.DataFrame(events)
display(events_df[["step", "event_type", "timestamp"]])

print("\nLast 2 events:")
for ev in events[-2:]:
    print(json.dumps(ev, indent=2, ensure_ascii=False))


Trace directory: /Users/admin/TuanDung/paper_implementation/plan_and_act_repro/data/raw/traces/nbtrace_20260219T093108Z
session exists: True
events exists: True

Session summary:
{
  "success": false,
  "step_count": 4,
  "llm_call_count": 9,
  "fallback_count": 0,
  "event_count": 23
}


,step,event_type,timestamp
0,0,planner_input,2026-02-19T09:31:09.940815+00:00
1,0,planner_output,2026-02-19T09:31:19.245508+00:00
2,1,executor_input,2026-02-19T09:31:19.247593+00:00
3,1,executor_output,2026-02-19T09:31:22.404951+00:00
4,1,environment_step,2026-02-19T09:31:24.026406+00:00
5,1,replanner_input,2026-02-19T09:31:24.027726+00:00
6,1,replanner_output,2026-02-19T09:31:40.847826+00:00
7,2,executor_input,2026-02-19T09:31:40.849111+00:00
8,2,executor_output,2026-02-19T09:31:47.306045+00:00
9,2,environment_step,2026-02-19T09:31:47.320600+00:00



Last 2 events:
{
  "run_id": "nbtrace_20260219T093108Z",
  "step": 4,
  "event_type": "replanner_output",
  "timestamp": "2026-02-19T09:32:49.839333+00:00",
  "payload": {
    "steps": [
      {
        "step_id": 1,
        "intent": "Use 'web_search' tool to find recent long-horizon LLM agent approaches related to Plan-and-Act.",
        "success_criteria": "3 recent long-horizon LLM agent approaches have been found."
      },
      {
        "step_id": 2,
        "intent": "Use 'fetch_url' tool to gather information about the first approach from the search results.",
        "success_criteria": "Information about the first approach has been gathered."
      },
      {
        "step_id": 3,
        "intent": "Use 'web_search' tool to find the second recent long-horizon LLM agent approach related to Plan-and-Act.",
        "success_criteria": "Second recent long-horizon LLM agent approach has been found."
      },
      {
        "step_id": 4,
        "intent": "Use 'fetch_url' tool 

## 7) Notes

- This notebook is intended for **full observability**, not minimal token usage.
- For repeatable runs, keep `OPENAI_MODEL` fixed and do not change prompt templates during comparison.
- You can copy this notebook pattern for other agent papers by replacing only:
  1. query/task
  2. toolset
  3. planner/executor/replanner prompt templates
